In [1]:
import tensorflow as tf
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.applications import InceptionV3

In [ ]:
idx = np.random.randint(1, 12000)

path = 'dataset/test/'
img = cv2.imread(path + 'Bean/0001.jpg', cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator  = train_datagen.flow_from_directory(
    'dataset/train/',
    target_size=(224, 224),
    # batch_size=256,
    class_mode='categorical',
    shuffle=False)

test_generator  = test_datagen.flow_from_directory(
    'dataset/test/',
    target_size=(224, 224),
    # batch_size=256,
    class_mode='categorical',
    shuffle=False)

valid_generator  = valid_datagen.flow_from_directory(
    'dataset/validation/',
    target_size=(224, 224),
    # batch_size=256,
    class_mode='categorical',
    shuffle=False)

In [ ]:
inc=InceptionV3(input_shape=(224,224,3),weights='imagenet',include_top=False)
inc.summary()

# 1. TRANSFER LEARNING

## 1.1 Normal

In [ ]:
for i in inc.layers:
  i.trainable = False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model_transf_learn_normal = Sequential()
model_transf_learn_normal.add(inc)
model_transf_learn_normal.add(layers.Flatten())
model_transf_learn_normal.add(layers.Dense(256, activation='relu'))
model_transf_learn_normal.add(layers.Dense(15, activation='softmax'))

model_transf_learn_normal.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

lr = 1e-3
opt = Adam(learning_rate=lr)

model_transf_learn_normal.compile(optimizer=opt,
                                  loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(
                                monitor = 'val_loss', 
                              mode = 'min', 
                              patience = 5,
                              restore_best_weights=True,
                              verbose = 1)

filepath = './model_transf_learn_normal.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]


In [ ]:
epochs = 500

print("[INFO]: Entrenando la red...")
model_tr_le_normal = model_transf_learn_normal.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=epochs,
        callbacks=callback_list)

## 1.2 weight regularization (Agregar una penalización en la funcion de perdida. OBJ: Limitar los valores grandes de los pesos del modelo)

In [32]:
from tensorflow.keras import regularizers

model_transf_learn_weight_regu = Sequential()
model_transf_learn_weight_regu.add(inc)
model_transf_learn_weight_regu.add(layers.Flatten())
model_transf_learn_weight_regu.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_transf_learn_weight_regu.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_transf_learn_weight_regu.compile(optimizer=Adam(learning_rate=lr),
                                       loss=tf.losses.CategoricalCrossentropy(),
                                        metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                                tf.keras.metrics.Precision(name='precision'),
                                                tf.keras.metrics.Recall(name='recall')])

In [34]:
filepath = './model_transf_learn_weight_regu.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_tr_le_we = model_transf_learn_weight_regu.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 1.3 drop out

In [ ]:
from tensorflow.keras.layers import Dropout

model_transf_learn_drop_out = Sequential()
model_transf_learn_drop_out.add(inc)
model_transf_learn_drop_out.add(layers.Flatten())
model_transf_learn_drop_out.add(layers.Dense(256, activation='relu'))
model_transf_learn_drop_out.add(layers.Dropout(0.5))
model_transf_learn_drop_out.add(layers.Dense(15, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model_transf_learn_drop_out.compile(optimizer=Adam(learning_rate=lr),
                                    loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [39]:
filepath = './model_transf_learn_drop_out.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_tr_le_drop = model_transf_learn_drop_out.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 1.4 batch normalization

In [41]:
from tensorflow.keras.layers import BatchNormalization

model_transf_learn_batch_norm = Sequential()
model_transf_learn_batch_norm.add(inc)
model_transf_learn_batch_norm.add(layers.Flatten())
model_transf_learn_batch_norm.add(layers.Dense(256, activation='relu'))
model_transf_learn_batch_norm.add(BatchNormalization())
model_transf_learn_batch_norm.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_transf_learn_batch_norm.compile(optimizer=Adam(learning_rate=lr),
                                      loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [43]:
filepath = './model_transf_learn_batch_norm.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_tr_le_batch_normal = model_transf_learn_batch_norm.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 1.5 Data augmentation

In [ ]:
train_datagen_data_aug = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=15, # grados de rotacion aleatoria
        width_shift_range=0.2, # fraccion del total (1) para mover la imagen
        height_shift_range=0.2, # fraccion del total (1) para mover la imagen
        horizontal_flip=True, # girar las imagenes horizontalmente (eje vertical)
        zoom_range=0.2 )# rango de zoom
valid_datagen_data_aug = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2)
test_datagen_data_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# Cargar las imágenes de entrenamiento utilizando flow_from_directory
train_generator_data_aug  = train_datagen_data_aug.flow_from_directory(
    'dataset/train/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator_data_aug  = test_datagen_data_aug.flow_from_directory(
    'dataset/test/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

valid_generator_data_aug  = valid_datagen_data_aug.flow_from_directory(
    'dataset/validation/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [46]:
model_transf_learn_data_augm = Sequential()
model_transf_learn_data_augm.add(inc)
model_transf_learn_data_augm.add(layers.Flatten())
model_transf_learn_data_augm.add(layers.Dense(256, activation='relu'))
model_transf_learn_data_augm.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_transf_learn_data_augm.compile(optimizer=Adam(learning_rate=lr),
                                     loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [48]:
filepath = './model_transf_learn_data_augm.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_tr_le_data_augm = model_transf_learn_data_augm.fit(
        train_generator_data_aug,
        validation_data=valid_generator_data_aug,
        callbacks = callback_list,
        epochs=epochs)

# 2. FINE TUNING

## 2.1 Normal

In [ ]:
for i in inc.layers:
  i.trainable = True

for layer in inc.layers:
  if layer.name == 'conv2d_80':
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada...')

In [ ]:
model_fine_tuning_normal = Sequential()
model_fine_tuning_normal.add(inc)
model_fine_tuning_normal.add(layers.Flatten())
model_fine_tuning_normal.add(layers.Dense(256, activation='relu'))
model_fine_tuning_normal.add(layers.Dense(15, activation='softmax'))

model_fine_tuning_normal.summary()

In [ ]:
model_fine_tuning_normal.compile(optimizer=Adam(learning_rate=lr),
                                 loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [53]:
filepath = './model_fine_tuning_normal.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_fi_tun_normal = model_fine_tuning_normal.fit(
        train_generator,
        validation_data=valid_generator,
        callbacks = callback_list,
        epochs=epochs)

## 2.2 weight regularization

In [55]:
model_fine_tuning_weight_regu = Sequential()
model_fine_tuning_weight_regu.add(inc)
model_fine_tuning_weight_regu.add(layers.Flatten())
model_fine_tuning_weight_regu.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_fine_tuning_weight_regu.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_fine_tuning_weight_regu.compile(optimizer=Adam(learning_rate=lr),
                                      loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [57]:
filepath = './model_fine_tuning_weight_regu.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_fi_tu_we = model_fine_tuning_weight_regu.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 2.3 drop out

In [ ]:
model_fine_tuning_drop_out = Sequential()
model_fine_tuning_drop_out.add(inc)
model_fine_tuning_drop_out.add(layers.Flatten())
model_fine_tuning_drop_out.add(layers.Dense(256, activation='relu'))
model_fine_tuning_drop_out.add(layers.Dropout(0.75))
model_fine_tuning_drop_out.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_fine_tuning_drop_out.compile(optimizer=Adam(learning_rate=lr),
                                   loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [61]:
filepath = './model_fine_tuning_drop_out.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_fi_tu_drop = model_fine_tuning_drop_out.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 2.4 batch normalization

In [63]:
model_fine_tuning_batch_norm = Sequential()
model_fine_tuning_batch_norm.add(inc)
model_fine_tuning_batch_norm.add(layers.Flatten())
model_fine_tuning_batch_norm.add(layers.Dense(256, activation='relu'))
model_fine_tuning_batch_norm.add(BatchNormalization())
model_fine_tuning_batch_norm.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_fine_tuning_batch_norm.compile(optimizer=Adam(learning_rate=lr),
                                     loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [65]:
filepath = './model_fine_tuning_batch_norm.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_fi_tu_batch_normal = model_fine_tuning_batch_norm.fit(
                      train_generator,
                      validation_data=valid_generator,
                      callbacks = callback_list,
                      epochs=epochs)

## 2.5 Data augmentation

In [67]:
model_fine_tuning_data_augm = Sequential()
model_fine_tuning_data_augm.add(inc)
model_fine_tuning_data_augm.add(layers.Flatten())
model_fine_tuning_data_augm.add(layers.Dense(256, activation='relu'))
model_fine_tuning_data_augm.add(layers.Dense(15, activation='softmax'))

In [ ]:
model_fine_tuning_data_augm.compile(optimizer=Adam(learning_rate=lr),
                                    loss=tf.losses.CategoricalCrossentropy(),
                                    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                                              tf.keras.metrics.Precision(name='precision'),
                                              tf.keras.metrics.Recall(name='recall')])

In [69]:
filepath = './model_fine_tuning_data_augm.hdf5'

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_loss', 
                                mode='min', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
print("[INFO]: Entrenando la red...")
model_fi_tu_data_augm = model_fine_tuning_data_augm.fit(
        train_generator_data_aug,
        validation_data=valid_generator_data_aug,
        callbacks = callback_list,
        epochs=epochs)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

models_info = [
    ("model_transf_learn_normal",        model_tr_le_normal),
    ("model_transf_learn_weight_regu",   model_tr_le_we),
    ("model_transf_learn_drop_out",      model_tr_le_drop),
    ("model_transf_learn_batch_norm",    model_tr_le_batch_normal),
    ("model_transf_learn_data_augm",     model_tr_le_data_augm),
    ("model_fine_tuning_normal",         model_fi_tun_normal),
    ("model_fine_tuning_weight_regu",    model_fi_tu_we),
    ("model_fine_tuning_drop_out",       model_fi_tu_drop),
    ("model_fine_tuning_batch_norm",     model_fi_tu_batch_normal),
    ("model_fine_tuning_data_augm",      model_fi_tu_data_augm),
]

def evaluate_models(models_info, test_generator):
    results = []

    y_true = test_generator.classes
    class_names = list(test_generator.class_indices.keys())

    for name, history in models_info:
        model = history.model
        epochs = len(history.epoch)

        y_pred_probs = model.predict(test_generator, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

        results.append({
            "Modelo": name,
            "Tipo": "Transfer" if "transf" in name else "Fine-tuning",
            "Técnica": name.split("_")[-1],
            "Épocas": epochs,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-score": f1
        })

    return pd.DataFrame(results)


tabla_resultados = evaluate_models(models_info, test_generator)
print(tabla_resultados.sort_values(by='F1-score', ascending=False).to_string(index=False))

In [ ]:
import pickle



with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_transf_learn_normal.pkl", 'wb') as file:
    pickle.dump(model_tr_le_normal.history, file)

path = "E:/00. Kaggle/18. Clasification Inception/Models"
model_transf_learn_normal.save(f"{path}/model_transf_learn_normal.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_transf_learn_weight_regu.pkl", 'wb') as file:
    pickle.dump(model_tr_le_we.history, file)

model_transf_learn_weight_regu.save(f"{path}/model_transf_learn_weight_regu.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_transf_learn_drop_out.pkl", 'wb') as file:
    pickle.dump(model_tr_le_drop.history, file)

model_transf_learn_drop_out.save(f"{path}/model_transf_learn_drop_out.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_transf_learn_batch_norm.pkl", 'wb') as file:
    pickle.dump(model_tr_le_batch_normal.history, file)

model_transf_learn_batch_norm.save(f"{path}/model_transf_learn_batch_norm.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_transf_learn_data_augm.pkl", 'wb') as file:
    pickle.dump(model_tr_le_data_augm.history, file)

model_transf_learn_data_augm.save(f"{path}/model_transf_learn_data_augm.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_fine_tuning_normal.pkl", 'wb') as file:
    pickle.dump(model_fi_tun_normal.history, file)

model_fine_tuning_normal.save(f"{path}/model_fine_tuning_normal.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_fine_tuning_weight_regu.pkl", 'wb') as file:
    pickle.dump(model_fi_tu_we.history, file)

model_fine_tuning_weight_regu.save(f"{path}/model_fine_tuning_weight_regu.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_fine_tuning_drop_out.pkl", 'wb') as file:
    pickle.dump(model_fi_tu_drop.history, file)

model_fine_tuning_drop_out.save(f"{path}/model_fine_tuning_drop_out.h5")


with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_fine_tuning_batch_norm.pkl", 'wb') as file:
    pickle.dump(model_fi_tu_batch_normal.history, file)

model_fine_tuning_batch_norm.save(f"{path}/model_fine_tuning_batch_norm.h5")

with open("E:/00. Kaggle/18. Clasification Inception/Models/historial_model_fine_tuning_data_augm.pkl", 'wb') as file:
    pickle.dump(model_fi_tu_data_augm.history, file)

model_fine_tuning_data_augm.save(f"{path}/model_fine_tuning_data_augm.h5")

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
# from tensorflow.keras.models import Model

# # Definir la entrada

# input_shape = (128, 128, 3)
# input_img = Input(shape=input_shape)

# # Encoder

# x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
# x = Conv2D(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# x = Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)

# # Bottleneck con dos caminos

# path1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# path1 = Conv2D(32, (3, 3), activation='relu', padding='same')(path1)
# path1 = Conv2D(1, (3, 3), activation='relu', padding='same')(path1)
# path1 = Conv2D(256, (3, 3), activation='relu', padding='same')(path1)

# path2 = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

# x = Concatenate()([path1, path2])

# # Decoder

# x = Conv2DTranspose(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# x = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# decoded_output = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# # Crear el modelo completo del autoencoder

# autoencoder = Model(input_img, decoded_output)

# # Compilar el modelo

# autoencoder.compile(optimizer='adam', loss='mse')

# # Resumen del modelo

# autoencoder.summary()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
# from tensorflow.keras.models import Model

# # Definir la entrada
# input_shape = (128, 128, 3)
# input_img = Input(shape=input_shape)

# # Encoder
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
# x = Conv2D(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# x = Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)

# # Decoder
# x = Conv2DTranspose(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# x = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
# decoded_output = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# autoencoder = Model(input_img, decoded_output)

# autoencoder.compile(optimizer='adam', loss='mse')

# autoencoder.summary()